In [1]:
from Dissertation.Func import Dissertation_Func_1D

In [2]:
from joblib import Parallel, delayed
import os
import numpy as np

failedruns4 = []

def save_results(model, base_dir="dissertation_func", save_stride=20):
    os.makedirs(base_dir, exist_ok=True)
    fname = os.path.join(base_dir, f"alpha_{model.alpha:.2f}_lambda_{model.k:.2f}_m0_{model.m0:.2f}_n0_{model.n0:.2f}.npz")

    indices = np.arange(0, len(model.times), save_stride)
    np.savez_compressed(fname,
                        x=model.x,
                        t=model.times[indices],
                        N_arr=model.N_arr[indices],
                        M_arr=model.M_arr[indices],
                        wave_speed=model.wave_speed)

def run_and_save_single(alpha, lam, m0, n0, base_dir="dissertation_func"):
    print(f"▶ Running: α = {alpha}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
    model = Dissertation_Func_1D(
        D=1.0, rho=1.0, K=1.0,
        k=lam, alpha=alpha,
        n0=n0, m0=m0,
        L=1000.0, N=5001,
        T=550, dt=0.1,
        scheme="AB2AM2",
        init_type="tanh",
        steepness=0.1
    )
    model.solve()
    speed, _, _ = model.estimate_wave_speed(plot=False)

    if speed is None:
        print(f"❌ Failed: α = {alpha}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
        return (alpha, lam, m0, n0, False)
    else:
        model.wave_speed = speed
        save_results(model, base_dir=base_dir)
        print(f"✅ Saved: α = {alpha}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
        return (alpha, lam, m0, n0, True)

def run_parallel_batch():
    alpha_vals  = [0.1, 0.3, 0.5, 1.0, 2, 5, 10]
    lambda_vals = [0.1, 0.2, 0.5, 0.8, 1.0, 2, 5, 10]
    m0_vals     = [0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1.0]
    n0_vals     = [0.1, 0.2, 0.5, 0.8, 0.9, 1.0]

    param_grid = [(alpha, lam, m0, n0) for alpha in alpha_vals for lam in lambda_vals for m0 in m0_vals for n0 in n0_vals]

    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(run_and_save_single)(alpha, lam, m0, n0) for alpha, lam, m0, n0 in param_grid
    )

    global failedruns4
    failedruns4 = [(alpha, lam, m0, n0) for (alpha, lam, m0, n0, success) in results if not success]
    if failedruns4:
        print("\n❌ The following runs failed to compute wave speed:")
        with open("failed_runs_dissertation_func.txt", "w") as f:
            for alpha, lam, m0, n0 in failedruns4:
                print(f"  α = {alpha}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
                f.write(f"alpha = {alpha}, lambda = {lam}, m0 = {m0}, n0 = {n0}\n")
    else:
        print("\n✅ All runs succeeded.")

if __name__ == "__main__":
    run_parallel_batch()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


▶ Running: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.1
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.2
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.9
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.1
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.8
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.2
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 1.0
Estimated speed = 0.2915, R² = 0.9793
Estimated speed = 0.2950, R² = 0.9799
Estimated speed = 0.2936, R² = 0.9804
Estimated speed = 0.2980, R² = 0.9770
Estimated speed = 0.2937, R² = 0.9783
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.5
Estimated speed = 0.3056, R² = 0.9803
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.8


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   55.1s


Estimated speed = 0.3022, R² = 0.9802
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.1
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.9
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.8
Estimated speed = 0.2904, R² = 0.9804
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.9
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 1.0
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.2
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.2
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 0.1
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.1
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 0.2
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 0.8
Estimated speed = 0.2955, R² = 0.9788
Estimated speed = 0.2921, R² = 0.9802
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 0.9


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.8min


...

[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 441.4min


Estimated speed = 0.2694, R² = 1.0000
✅ Saved: α = 10, λ = 2, m₀ = 0.9, n₀ = 0.5
▶ Running: α = 10, λ = 2, m₀ = 0.95, n₀ = 0.9
Estimated speed = 0.2694, R² = 1.0000
✅ Saved: α = 10, λ = 2, m₀ = 0.9, n₀ = 0.8
▶ Running: α = 10, λ = 2, m₀ = 0.95, n₀ = 1.0
Estimated speed = 0.2694, R² = 1.0000
✅ Saved: α = 10, λ = 2, m₀ = 0.9, n₀ = 0.9
▶ Running: α = 10, λ = 2, m₀ = 1.0, n₀ = 0.1
Estimated speed = 0.2694, R² = 1.0000
Estimated speed = 0.2694, R² = 1.0000
✅ Saved: α = 10, λ = 2, m₀ = 0.95, n₀ = 0.1
▶ Running: α = 10, λ = 2, m₀ = 1.0, n₀ = 0.2
✅ Saved: α = 10, λ = 2, m₀ = 0.9, n₀ = 1.0
▶ Running: α = 10, λ = 2, m₀ = 1.0, n₀ = 0.5
Estimated speed = 0.2694, R² = 1.0000
✅ Saved: α = 10, λ = 2, m₀ = 0.95, n₀ = 0.2
▶ Running: α = 10, λ = 2, m₀ = 1.0, n₀ = 0.8
Estimated speed = 0.2694, R² = 1.0000
✅ Saved: α = 10, λ = 2, m₀ = 0.95, n₀ = 0.5
▶ Running: α = 10, λ = 2, m₀ = 1.0, n₀ = 0.9
Estimated speed = 0.2694, R² = 1.0000
✅ Saved: α = 10, λ = 2, m₀ = 0.95, n₀ = 0.8
▶ Running: α = 10, λ = 2, m₀ = 

[Parallel(n_jobs=-1)]: Done 2649 tasks      | elapsed: 449.1min


Estimated speed = 0.4729, R² = 1.0000
✅ Saved: α = 10, λ = 10, m₀ = 0.1, n₀ = 0.8
▶ Running: α = 10, λ = 10, m₀ = 0.2, n₀ = 1.0
Estimated speed = 0.4729, R² = 1.0000
✅ Saved: α = 10, λ = 10, m₀ = 0.1, n₀ = 0.9
▶ Running: α = 10, λ = 10, m₀ = 0.5, n₀ = 0.1
Estimated speed = 0.4729, R² = 1.0000
✅ Saved: α = 10, λ = 10, m₀ = 0.1, n₀ = 1.0
▶ Running: α = 10, λ = 10, m₀ = 0.5, n₀ = 0.2
Estimated speed = 0.4729, R² = 1.0000
✅ Saved: α = 10, λ = 10, m₀ = 0.2, n₀ = 0.1
▶ Running: α = 10, λ = 10, m₀ = 0.5, n₀ = 0.5
Estimated speed = 0.4729, R² = 1.0000
Estimated speed = 0.4729, R² = 1.0000
✅ Saved: α = 10, λ = 10, m₀ = 0.2, n₀ = 0.2
▶ Running: α = 10, λ = 10, m₀ = 0.5, n₀ = 0.8
✅ Saved: α = 10, λ = 10, m₀ = 0.2, n₀ = 0.5
▶ Running: α = 10, λ = 10, m₀ = 0.5, n₀ = 0.9
Estimated speed = 0.4729, R² = 1.0000
✅ Saved: α = 10, λ = 10, m₀ = 0.2, n₀ = 0.8
▶ Running: α = 10, λ = 10, m₀ = 0.5, n₀ = 1.0
Estimated speed = 0.4729, R² = 1.0000
✅ Saved: α = 10, λ = 10, m₀ = 0.2, n₀ = 0.9
▶ Running: α = 10, λ =

[Parallel(n_jobs=-1)]: Done 2688 out of 2688 | elapsed: 452.9min finished


<h2> Lambda Test

In [ ]:
from joblib import Parallel, delayed
import os
import numpy as np

failedruns4_lambda = []

def save_results(model, base_dir="dissertation_func_lambda_check", save_stride=150):
    os.makedirs(base_dir, exist_ok=True)
    fname = os.path.join(base_dir, f"alpha_{model.alpha:.2f}_lambda_{model.k:.2f}_m0_{model.m0:.2f}_n0_{model.n0:.2f}.npz")

    indices = np.arange(0, len(model.times), save_stride)
    np.savez_compressed(fname,
                        x=model.x,
                        t=model.times[indices],
                        N_arr=model.N_arr[indices],
                        M_arr=model.M_arr[indices],
                        wave_speed=model.wave_speed)

def run_and_save_single(alpha, lam, m0, n0, base_dir="dissertation_func_lambda_check"):
    print(f"▶ Running: α = {alpha}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
    
    model = Dissertation_Func_1D(
        D=1.0, rho=1.0, K=1.0,
        k=lam, alpha=alpha,
        n0=n0, m0=m0,
        L=100.0, N=10001,
        T=550, dt=0.01,
        scheme="AB2AM2",
        init_type="tanh",
        steepness=0.1
    )
    
    model.solve()
    speed, _, _ = model.estimate_wave_speed(plot=False)
    model.wave_speed = speed  # may be None — still save

    save_results(model, base_dir=base_dir)

    if speed is None:
        print(f"⚠️  Speed failed: α = {alpha}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
        return (alpha, lam, m0, n0, False)
    else:
        print(f"✅ Saved: α = {alpha}, λ = {lam}, m₀ = {m0}, n₀ = {n0}, c = {speed:.3f}")
        return (alpha, lam, m0, n0, True)

def run_parallel_batch():
    alpha_vals  = [0.01, 0.1, 0.3, 0.5, 1.0, 2, 5, 10, 50, 100]
    lambda_vals = [0.01, 0.1, 0.2, 0.5, 0.8, 1.0, 2, 5, 10, 50, 100]
    m0_vals     = [0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1.0]
    n0_vals     = [1.0]

    param_grid = [(alpha, lam, m0, n0) for alpha in alpha_vals for lam in lambda_vals for m0 in m0_vals for n0 in n0_vals]

    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(run_and_save_single)(alpha, lam, m0, n0) for alpha, lam, m0, n0 in param_grid
    )

    global failedruns4_lambda
    failedruns4_lambda = [(alpha, lam, m0, n0) for (alpha, lam, m0, n0, success) in results if not success]

    if failedruns4_lambda:
        print("\n❌ Some runs completed but failed to compute wave speed:")
        with open("failed_runs_dissertation_func_lambda_check.txt", "w") as f:
            for alpha, lam, m0, n0 in failedruns4_lambda:
                print(f"  α = {alpha}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
                f.write(f"alpha = {alpha}, lambda = {lam}, m0 = {m0}, n0 = {n0}\n")
    else:
        print("\n✅ All runs computed wave speed successfully.")

if __name__ == "__main__":
    run_parallel_batch()